In [10]:
import cv2
import numpy as np
#import matplotlib.pyplot as plt
import cv2 as cv

In [11]:
line_global=0
def make_coordinates(image,line_parameters):
    global line_global
    #print(line_global)
    try:
        slope, intercept = line_parameters
        line_global=line_parameters
    except:
        slope, intercept=line_global
    #print(slope,intercept)
    #print("-----------------------------")
    y1 = image.shape[0]  # Height
    y2 = int(y1 * (3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1,y1,x2,y2]) 


In [12]:
def average_slope_intercept(image,lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2),(y1,y2),1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope<0:
            left_fit.append((slope,intercept))
        else :
            right_fit.append((slope,intercept))
    left_fit_average = np.average(left_fit,axis=0)
    right_fit_average = np.average(right_fit,axis=0)
    left_line = make_coordinates(image,left_fit_average)
    right_line = make_coordinates(image,right_fit_average)
    return np.array([left_line,right_line])


In [13]:
def canny(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)   # Kernel size is 5x5
    canny = cv2.Canny(blur,50,150)
    return canny

In [14]:

def display_lines(image,lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)  # Reshaping all the lines to a 1D array.
            #print("-",x1,"-",y1,"-",x2,"-",y2,"-")
            try:
                line_image=cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5) # Draw a Blue Line(BGR in OpenCV)
            except:
                print("except")
    return line_image

In [15]:

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
        [(200,height),(1100,height),(550,250)]
    ])          # Triangle polygon because cv2.fillPoly expects an array of polygons.
    mask = np.zeros_like(image)   # Create a black mask to apply the above Triangle on.
    cv2.fillPoly(mask,polygons,255)     # A complete white triangle polygon on a black mask.
    masked_image = cv2.bitwise_and(image,mask)
    return masked_image

In [16]:
""" image = cv2.imread('./test_image.jpg')
lane_image = np.copy(image)     # Always make a copy when working with arrays rather than directly assigning
canny_image = canny(lane_image)
cropped_image = region_of_interest(canny_image)
lines = cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=10,maxLineGap=5)
averaged_lines = average_slope_intercept(lane_image,lines)
line_image = display_lines(lane_image,averaged_lines)
combo_image = cv2.addWeighted(lane_image,0.8,line_image,1,1)    # Imposing the line_image on the original image

cv2.imshow('Result',combo_image)
cv2.waitKey(0) """
fourcc = cv2.VideoWriter_fourcc(*'MP4V') # XVID algoritmasını tanımlama
kayit = cv2.VideoWriter('videolar/kayit.mp4',fourcc,20.0,(640,480))

In [17]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1280,720))
cap = cv2.VideoCapture("test_video.mp4")
while(cap.isOpened()):
    try:
        _, frame = cap.read()
        canny_image = canny(frame)
        cropped_image = region_of_interest(canny_image)
        lines = cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=10,maxLineGap=5)
        averaged_lines = average_slope_intercept(frame,lines)
        line_image = display_lines(frame,averaged_lines)
        #cukur
        gray1 = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        #_, threshold1 = cv.threshold(gray1, 200, 255, cv.THRESH_BINARY)
        _, threshold1 = cv.threshold(gray1, 79, 84, cv.THRESH_BINARY)
        _,contours1 ,_x = cv.findContours(threshold1, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        for con in contours1:
            vertices1 = cv.approxPolyDP(con, 0.01 * cv.arcLength(con, True), True)
            vertices2 = cv.approxPolyDP(con, 0.01 * cv.arcLength(con, False), True)
            if len(vertices1) > 9 and cv.arcLength(con, True) < 500 and len(vertices2) > 9:
                cv.drawContours(frame, [con], 0, (0, 0, 255), 3)
        combo_image = cv2.addWeighted(frame,0.8,line_image,1,1)    # Imposing the line_image on the original image
        a=type(combo_image)
        #print(combo_image.shape)
        out.write(combo_image) # video yazmayı başlatma
        
        #cv2.imshow('Result',combo_image)
        cv2.waitKey(1)
    except:
        break
cap.release()

out.release()

cv2.destroyAllWindows()